In [25]:

import matplotlib.pyplot as plt
import numpy as np
import cv2
%matplotlib inline

In [26]:
import glob
#Data folders
glaucoma_dir ='E:\\glaucoma_final\\dataset\\g'
healthy_dir = 'E:\\glaucoma_final\\dataset\\healthy'
# images are divided up into vehicles and non-vehicles
glaucoma = []
healthy = []
# Read 
images = glob.iglob(glaucoma_dir + '/**/*.png', recursive=True)

for image in images:
        glaucoma.append(image)
        
images = glob.iglob(healthy_dir + '/**/*.jpg', recursive=True)

for image in images:
        healthy.append(image)

        
print(len(glaucoma))
print(len(healthy))

56
56


In [27]:
def Image_preprocessing(images):
    preprocessed_image = []
    for image in images:
        image = cv2.imread(image)
        reshape = cv2.resize(image, (400, 400))
        blue, green, red = cv2.split(reshape)
        equ = cv2.equalizeHist(green)
        preprocessed_image.append(equ)
    return preprocessed_image

In [28]:
glaucoma_preprocessing = Image_preprocessing(glaucoma)
print(len(glaucoma_preprocessing))

56


In [29]:
healthy_preprocessing = Image_preprocessing(healthy)
print(len(healthy_preprocessing))

56


In [30]:
def brief_feature_extraction(images):
    brief_feature_extracted = []
    for image in images:
        image_features = []
        feature_image = np.copy(image)
        brief_features = []
        fast = cv2.FastFeatureDetector_create()
        brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
        keypoints = fast.detect(feature_image, None)
        keypoints_brief, descriptors = brief.compute(feature_image, keypoints)
        brief_features.append(descriptors[0:200])
        brief_features = np.ravel(brief_features)
        image_features.append(brief_features)
        brief_feature_extracted.append(np.concatenate(image_features))
    return brief_feature_extracted

In [31]:
glaucoma_feature = brief_feature_extraction(glaucoma_preprocessing)
print(len(glaucoma_feature))

56


In [32]:
healthy_feature = brief_feature_extraction(healthy_preprocessing)
print(len(healthy_feature))

56


In [33]:
X = np.vstack((glaucoma_feature,healthy_feature)).astype(np.float64)

In [34]:
print(X.shape)

(112, 6400)


In [35]:
y = np.hstack((np.zeros(len(glaucoma_feature)), np.ones(len(healthy_feature))))

In [36]:
from sklearn.svm import LinearSVC

In [37]:
svc = LinearSVC()
svc.fit(X, y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [38]:
from sklearn.metrics import confusion_matrix
y_pred = svc.predict(X)
print(confusion_matrix(y, y_pred))

[[56  0]
 [ 0 56]]


In [39]:
import pickle
filename = 'E:\\glaucoma_final\\finalized_model_brief_svc.sav'
pickle.dump(svc, open(filename, 'wb'))